In [95]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
import warnings

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics

nastavení

In [96]:
graf_priprava_dat = False

df_path = fr"C:\Users\halik\OneDrive\Dokumenty\GitHub\predikce-nabidky\production code\production dataset.xlsx"#@ cesta k datasetu
parametry = ['aktivni-1', 'nove-1', 'zbytek-1'] #@ parametry pouzite v modelu


#@ hyperparametry ARIMY 
p = 2  # AR myslim si ze by to melo byt 2 aby nebyl overfitting protoze pacf hodne spadne po lag=2
d = 1  # I 
q = 2  # MA


import dat

In [97]:

#@ args:
    # remove_first: odstrani prvních remove first řádků z datasetu (protože nejsou kompletni...)
    # remove_last: odstrani posledni remove_last řádků proto z data setu 
        # default = 6 (momentalne je poslednich 6 radku na predikce)
def get_df(remove_first = 0, remove_last = 6):
    df = pd.read_excel(df_path)
    df = df.drop(df.index[:remove_first])
    if remove_last != 0:
        df = df.drop(df.index[-remove_last:])
    df['datetime'] = pd.date_range(start='2016-03-01', periods=len(df.index.values), freq='MS')
    return df

detrendování dat

In [98]:

df = get_df(12)
df_detrend = df
columns = ['aktivni', *parametry]
for column in columns:
    df_detrend[column] = pd.Series((df[column]))

deseasonalizace dat

In [99]:
df_deseasoned = df_detrend.copy()
for parametr in parametry:
    decomp = sm.tsa.seasonal_decompose(df_detrend[parametr], period=12, model='additive')
    df_deseasoned[parametr] = df_detrend[parametr] - decomp.seasonal.values

arima modely

    rolling window forecast

In [100]:
# Lists to store predictions and actual values for the current window size
predictions = []
actual_values = []
#! smazat na debuging
warnings.filterwarnings("ignore")
#@ parametres
rw_size = 56
parametry = ['nove-1', 'aktivni']
df_arima = df_deseasoned[parametry]
external_regressor_list = ['nove-1']
# predikce pomoci rw forecastu

# Select training data
df_train = df_arima

# Fit ARIMA model
model = ARIMA(df_train[['aktivni']], order=(p, d, q), exog=df_train[[*external_regressor_list]])
model = model.fit()



In [101]:
# Predict using the model
df_new = get_df(remove_last = 0) # 0 protze nechci odebrat radek co predpovidam
df_new = df_new.drop(df_new.index[0:-1])
df_pred = model.forecast(exog=df_new[[*external_regressor_list]])
print(f"predikce na dalsi mesic ({int(df_new['datum'])}) je {int(df_pred)} bytu v nabidce")


predikce na dalsi mesic (202312) je 8545 bytu v nabidce
